## Chapter 32
# Properties of Time Series 


## Introduction

Time series are an extremely common data type. Just a few of the many applications of time series analysis include:

- **Demand forecasting:** Electricity production, Internet bandwidth, Traffic management, Inventory management
- **Medicine:** Time-dependent treatment effects, EKG, EEG
- **Engineering and Science:** Signal analysis, Analysis of physical processes
- **Capital markets and economics:** Seasonal unemployment, Price/return series, Risk analysis

In this chapter, we will explore some fundamental properties of time series data, in particular:  

- How time series data differ from non-time ordered data.      
- Basic properties of time series:      
  - Correlation properties of time series.      
  - Properties of random walks.  
  - Time series with trends.    
  - Seasonal properties of time series.   
- How to perform and understand the decomposition of time series.    

****
**Resources:** Here is a selection of resources you can use to go deeper into time series analysis with Python:

1. If you would like more information and examples for working with time series data in [Python Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/timeseries.html).  
2. The Python [pmdarima package user quide](https://alkaline-ml.com/pmdarima/user_guide.html) provides some hands-on tutorial material.    
3. The Python [KATS time series package](https://facebookresearch.github.io/Kats/) provides a number of state-of-the-art time series methods and models.  
4. The go-to book is *Forecasting, Principles and Practice*, 2nd ed. by Rob J Hyndman and George Athanaspoulos, and is available as an [online book](https://otexts.com/fpp2/).   
5. [Darts](https://unit8co.github.io/darts/index.html), a newly released Python time series package combines state-of-the-art linear models with some powerful deep learning models.  
****

## Short History of Time Series Analysis

The history of time series analysis starts with the pioneering work of George Udny Yule (1927) and Gilbert Walker (1931). Both Yule and Walker developed auto-regressive (AR) models for stochastic time series.

<img src="../images/George_Udny_Yule.jpg" alt="Drawing" style="width:200px; height:250px"/>
<center>George Yule; time series pioneer</center>

Mathematical prodigy, Norbert Weiner, invented filters for stochastic time series processes during the Second World War. Weiner worked at MIT and was assigned to a project to improve the accuracy of anti-aircraft guns using the noisy radar signals of the day. He published his seminal paper on the subject in 1949. If you have recently used a mobile phone or streamed video or audio you are benefiting from Wiener's research!

<img src="../images/Norbert_wiener.jpg" alt="Drawing" style="width:200px; height:250px"/>
<center>Norbert Weiner: Invented time series filter</center>

The exponential smoothing methods of time series analysis and forecasting were first published by Robert Goodell Brown in 1956, and Charles C. Holt in 1957.

George Box and Gwilym Jenkins fully developed a statistical theory  of time series by extending the work of Yule and Walker in the 1950s and 1960s. This work was fully developed in their seminal 1970  book. Their theory included the autoregressive moving average (ARMA) model and the autoregressive integrated moving average (ARIMA) models we use in this notebook.

George Box was married to Joan Fisher Box, an outstanding statistician in her own right and daughter of Ronald Fisher. 

<img src="../images/GeorgeEPBox.jpg" alt="Drawing" style="width:200px; height:250px"/>
<center>George Box fully developed the ARIMA model</center>

<img src="../images/BoxJenkins.jpg" alt="Drawing" style="width:175px; height:250px"/>
<center>Seminal book: by Box and  Jenkins</center>


## How Are Time Series Data Different?

Time series data are different from the data types we have encountered so far. Up until now, we have been able to construct models using the assumption of **independent and identically distributed (iid) errors**. When sampling such data order does not matter.

However, for time series data, the observations are ordered by time and the observed values are **serially correlated**. For serially correlated data, the value of an observation will depend on one or more of the previous observations. Just a few of the many examples of data exhibiting serial correlation of the values include      
- Temperature forecasts, where the future values are correlated with the current values. 
- The opening price of a stock is correlated with the price at the previous close. 
- The daily sales volume of a product is correlated with the previous sales volume. 
- A medical patient's blood pressure reading is correlated with the previous observations.   

Given the serial correlation common in time series data, we will need models which account for this behavior. In this lesson our focus will be on a class of models known as **autoregressive integrative moving average (ARIMA) models**. These are linear models which account for the serial correlation in time series data.     

> **Note:** In this lesson we will work with the time series tools available in the Python Pandas, [statsmodels](http://www.statsmodels.org/stable/user-guide.html#time-series-analysis) and [pmdariama](https://alkaline-ml.com/pmdarima/) packages. However, the state-of-the-art time series models are often found in R packages. You can find considerable information on state-of-the-art time series forecasting in the blogs, R packages and books on time series with R on [Rob Hyndman's](https://robjhyndman.com/) web site. 



## Working with Time Series in Pandas

The Pandas package has significant capabilities for manipulation of time series data. The key to working with time series data is the index of the Pandas series or data frame. The index contains the [date-time information](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html) or [time delta (interval) information](https://pandas.pydata.org/pandas-docs/stable/user_guide/timedeltas.html). 

Let's start with a simple univariate time series example. A univariate time series can be represented as a Pandas series, with the appropriate index. 

> **Note:** The use of a [Pandas Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) object to hold a univariate time series may seem redundant or confusing. The time series is a time-ordered sequence of data values. Whereas, the Pandas Series object is a univariate data structure. A Pandas Series object contains time series data when an appropriate time series index is used. 

The code in the cell below creates a series of sinusoidal values. Execute this code and examine the results.

In [ ]:
from math import sin
import pandas as pd
import numpy as np
import numpy.random as nr
from math import pi
from scipy.stats import zscore
import sklearn.linear_model as lm
import statsmodels.tsa.seasonal as sts
import scipy.stats as ss
import statsmodels.tsa.arima_process as arima
from statsmodels.tsa.arima.model import ARIMA, ARIMAResults
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing
from statsmodels.stats.diagnostic import acorr_ljungbox
import statsmodels.api as sm
import pmdarima as pm
import statsmodels.graphics.tsaplots as splt
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 

%matplotlib inline

ts = pd.Series([sin(x/30.0) for x in range(366)])
ts.head(20)

The object you created is a series of floating point values. However, this is not yet a time series object, since there is no time indexing. For that, we need to add a time index or time difference index.

The code in the cell below adds a new set of index values to the Pandas Series object. Execute this code and examine the results. 

In [ ]:
ts.index = pd.date_range(start = '1-1-2016', end = '12-31-2016', freq = 'D')
ts.head(20)

You can see that the index is now the date-time for each value. This Pandas series is now an actual time series. 

Let's plot the time series. The code in the cell below plots the values of the time series against the index. Notice that there is no need to explicitly specify the values for the x-axis as these values are implied by the index of the Pandas Series object. Execute the code and examine the results. 

In [ ]:
_,ax = plt.subplots(figsize=(8,3))
ts.plot(ax=ax)
ax.set_title('A simple time series plot')
ax.set_ylabel('Value')
ax.set_xlabel('Date')

You can see that the time axis is labeled automatically. 

Pandas provides many methods to [manipulate and transform time series](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html). For example, one can subset a time series using a range of time values from the index. The code in the cell below takes a subset of the time series by specifying a date range and displaying a plot. Execute this code and examine the result. 

In [ ]:
_,ax = plt.subplots(figsize=(8,3))
ts['1/1/2016':'6/30/2016'].plot(ax=ax)
ax.set_title('A simple time series plot')
ax.set_ylabel('Value')
ax.set_xlabel('Date')

Notice that the plot covers the subset of dates specified. 

## Basic Time Series Properties

In this section, we will explore some basic properties of time series. Understanding these properties will help you understand the fundamental properties of time series, including    
1. Random **white noise series** are time series of random uncorrelated values. The is no predictive information in a white noise series.    
2. The **correlation** properties of time series, differentiate these data from unordered data. We can deduce several important properties of time series from the study of their correlation properties.   
3. The concept of **stationarity** is a foundation on which many time series models rely. In simple terms, a stationary time series has statistical properties that are invariant in time.  
3. Time series with **random walks** have drift  
4. Long-term **trend** is a common property of time series.    

### Properties of White Noise Series

A random series is **independent identically distributed (iid)** noise drawn from a Normal distribution. Such a series is said to be a **white noise** series. Since the series is iid there is no correlation from one value to the next. We can write a **discrete** white noise time series as just:

$$X(t) = (w_1, w_2, w_3, \dots, w_n)\\
where\\
w_t \sim \mathbb{N}(0, \theta)$$

Notice that the standard deviation and therefore the variance of the series, $\theta$, is constant in time. We say that a time series with a constant variance is **stationary**. The statistical properties of a stationary time series do not vary with time. 

Further, the values of the time series are given at specific or discrete times, making this a **discrete time series**. In computational time series analysis, we nearly always work with discrete time series. Some time series are inherently discrete including, the unemployment rate average over a month, and the daily closing price of a stock. Even if the underlying time series is continuous, we typically work with **values sampled at discrete points in time**. For example, temperature is a continuous variable, but we will generally work with sampled values, such as hourly measurements. 

The code in the cell below creates a time series from an iid Normal distribution with mean zero. Execute this code and note the attributes and the plot.

In [ ]:
def plot_ts(ts, lab = ''):
    _,ax = plt.subplots(figsize=(9,4))
    ts.plot(ax=ax)
    ax.set_title('Time series plot of ' + lab)
    ax.set_ylabel('Value')
    ax.set_xlabel('Date')

nr.seed(3344)
white = pd.Series(nr.normal(size = 730),
                 index = pd.date_range(start = '1-1-2014', end = '12-31-2015', freq = 'D'))
plot_ts(white, 'white noise')

Notice that the values of the time series seem to wander randomly around zero, with no particular trend. 

Next, let's look at the distribution of the time series values. The code in the cell below plots the histogram and Q-Q Normal plot of the values of the time series. Run this code and examine the results.  

In [ ]:
def dist_ts(ts, lab = '', bins = 40):
    _,ax = plt.subplots(1,2,figsize=(10,4))
    ## Plot the histogram with labels
    ts.hist(ax = ax[0], bins = bins, alpha = 0.5);
    ax[0].set_xlabel('Value');
    ax[0].set_ylabel('Frequency');
    ax[0].set_title('Histogram of ' + lab);
    ## Plot the q-q plot on the other axes
    ss.probplot(ts, plot = ax[1]);

dist_ts(white, 'white noise')    

As expected, the values of the white noise series are Normality distributed. When examining these plots keep in mind there are only 365 values, so we should expect quite a lot of random variation. 

### Autocorrelation of White Noise Series

The values of the white noise series are iid, so we do not expect the values to show any dependency over time. In time series analysis we measure dependency using **autocorrelation**. Autocorrelation is the correlation of a series with itself **lagged** (offset in time) by some number of time steps. Autocorrelation at lag k can be written as    

$$\rho_k = \frac{\gamma_k}{n \sigma^2} = \frac{1}{n \sigma^2} \sum_{t = 1}^N (y_{t} - \mu) \cdot (y_{t-k} - \mu)$$      
Where:   
\begin{align}
y_t &= observation\ at\ time\ t\\ 
k &= lag\\
\gamma_k &= covariance\ lag\ k\\
\mu &= mean\ of\ the\ series\\ 
\sigma^2 &= variance\ of\ the\ series = \frac{1}{n-1}\Sigma_{t = 1}^N (y_{t} - \mu) \cdot (y_{t} - \mu)
\end{align}

Notice that for any series, $\rho_0 = 1$. In other words, the autocorrelation of a series at **lag zero** equals 1.0. 
T
We can also define a second order **partial autocorrelation**. The Partial autocorrelation at lag k is the correlation that results from removing the effect of any correlations due to the terms at smaller lags.

Let's plot the autocorrelation function (acf), using [statsmodels.graphics.tsaplots.plot_acf](https://www.statsmodels.org/stable/generated/statsmodels.graphics.tsaplots.plot_acf.html), and partial autocorrelation function (pacf), using [statsmodels.graphics.tsaplots.plot_pacf](https://www.statsmodels.org/stable/generated/statsmodels.graphics.tsaplots.plot_pacf.html), of the white noise series. Run the code in the cell below to compute and plot these functions. 

In [ ]:
def auto_partial_corr_plot(ts):
    _,ax = plt.subplots(2,1, figsize=(8,8))
    _=splt.plot_acf(ts, lags = 40, ax=ax[0]);
    _=splt.plot_pacf(ts, lags = 40, method='yw', ax=ax[1]);

auto_partial_corr_plot(white)

As expected the white noise series only has significant autocorrelation and partial autocorrelation values at lag zero. There are no significant partial autocorrelation values. The shaded blue area on these plots shows the **95% confidence interval**. 

****
**Note:** The Python statsmodels package uses the engineering convention for displaying partial autocorrelation. The value at 0 lag, which always must be 1.0, is displayed. In many statistical packages, including R, this 0 lag value is not displayed. This difference in conventions can lead to a lot of confusion. 
****

### Hypothesis tests on autocorrelation      

Visually, we can see that the ACF and PACF for the white noise series have values largely within the 95% confidence interval of zero. While visual inspection is always a good idea, in some marginal cases, it will be worth doing a formal hypothesis test to determine if a series is in fact white noise. These tests are performed on the distribution of the ACF values over some number of lags. This distribution turns out to be approximate $\chi^2$ making the compuation of p-values simple. 

There are a number of such tests, several of which are supported in the [Statsmodels Regression Diagnostics and Specification Tests](https://www.statsmodels.org/dev/diagnostic.html#autocorrelation-tests) packages. Here, we will focus on the [Ljung-Box Q statistic](https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.acorr_ljungbox.html#statsmodels.stats.diagnostic.acorr_ljungbox), which is known to have a distribution that is a good approximatiion to the $\chi^2$ distribution. The Ljung-Box Q statistic is based on autocorrelation of at multiple lag values, $\rho_i$. The null hypothesis is that there is no serial correlation between the values, and that the values are independently distributed. The alternative hypothesis is that there is serial correlation giving high values of the Ljung-Box Q statistic and a small p-value. The Ljung-Box Q statistic is computed from the following relationship:           

$$Q = n(n+2) \sum_{k=1}^h \frac{\hat{\rho}^2_2}{n-k}$$

> **Exercise 32-1:** In the cell below create and execute code to do the following:    
> 1. Create a monthly series `(freq = 'M')` of white noise plus 2.0 times a sinusoidal component, with a 12-month period. Use a date range from January 2001 to January 2016. Name your series `ts_periodic`. The white noise component should have a mean of 0 and a standard deviation of 1.0. **Hint:** You will need to use the `pi` constant from the `math` library. The sinusoidal component will be `2 * numpy.sin(pi*x/6)`. Use `numpy.random.seed(6677)` to set the random number seed.    
> 2. Create a time series plot of the result. Then plot the acf and pacf of the time series for 40 lags.   

In [ ]:
## Put you code below   





> 3. Apply the [acorr_ljungbox](https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.acorr_ljungbox.html) function to the white noise series and the periodic time series with `lags=40` and `model_df=0`.     

In [ ]:
## Put your code below




> Examine these results and answer the following questions:     
> 1. What do the ACF and PACF tell you about the periodicity of the series?      
> 2. Does the significance of the periodic series of the acf and pacf change with lag?     
> 3. Examine Ljung-Box Q statistic and p-values for the white noise series and the period series. What can you say about the significance of serial correlation for these two series?    

> **Answers:**  
> 1.                     .     
> 2.           
> 3.             

### Random Walks

A **random walk** is defined by the sum of a white noise series. Since the random walk is the sum of all previous white noise terms, we say a random walk is **integrative**. In other words, the value of the random walk is the **cumulative sum of the preceding white noise series**. 

$$y_t = y_{t-1} + w_t$$

Where the $w_t$ is Normally distributed with variance $\sigma$: 

$$w_t \sim \mathbb{N}(0,\sigma)$$

The **innovations** of the random walk, $w_t$, are the **first order differences** of the time series values:   

$$w_t = y_t - y_{t-1}$$ 

Or, with a little bit of algebra, it is easy to show that:    

$$y_t = y_0 + w_t + \sum_{i=1}^{t-1} w_i$$

Where $y_0$ is an initial value.   

The forgoing sum accumulates over the history of the time series. This property leads to the term **integrative** being used for random walk components of time series. This is an example of a **stochastic process**, having random behavior over time.    

But note that the covariance of a random walk increases with time and is not bounded.

$$\gamma_k = Cov(y_t, y_{t+k}) = t \sigma^2 \rightarrow \infty\ as\ t \rightarrow \infty$$

Therefore, the random walk is **not stationary** . 

The code in the cell below computes a random walk series by taking the cumulative sum over Normally distributed innovations. Run this code and examine the results. 

In [ ]:
nr.seed(3344)
def ran_walk(start = '1-1990', end = '1-2015', freq = 'M', sd = 1.0, mean = 0):
    dates = pd.date_range(start = start, end = end, freq = freq)
    walk = pd.Series(nr.normal(loc = mean, scale = sd, size = len(dates)),
                    index = dates)
    return(walk.cumsum())
walk = ran_walk()   
plot_ts(walk, 'random walk')

The random walk wanders back and forth somewhat randomly. A random walk may appear to follow a trend, but there is no reason to believe the trend will continue. 

> **Exercise 32-2:** What does the distribution of values of the random walk time series look like? What about the ACF and PACF of the random walk? In the cell below, create and execute the code to examine the probability distribution, ACF and PACF of the random walk. 

In [ ]:
## Put your code below



> Examine these results and answer the following questions:    
> 1. How close to Normally distributed are the values of the random walk?    
> 2. How are the properties of the ACF and PACF different from those of the white noise series and why?    
> 3. How can you explain the difference between the ACF and PACF in terms of the serial correlation properties of a random walk?    

> **Answers:**   
> 1.           
> 2.          
> 3.         

### White Noise Series with Trend

What happens when we add a trend to the white noise series? Run the code in the cell below which adds a linear trend to a white noise series. 

In [ ]:
nr.seed(6677)
def trend(start = '1-1990', end = '1-2015', freq = 'M', slope = 0.02, sd = 0.5, mean = 0):
    dates = pd.date_range(start = start, end = end, freq = freq)
    trend = pd.Series([slope*x for x in range(len(dates))],
                    index = dates)
    trend = trend + nr.normal(loc = mean, scale = sd, size = len(dates))
    return(trend)
                              
trends = trend()   
plot_ts(trends, 'white noise with trend')

As expected, the time series trends upward with a linear trend with iid Normal deviations. 

Run the code in the cell below to examine the distribution of values in this time series.

In [ ]:
dist_ts(trends, lab = '\n trend + white noise')

The distribution shows some skew when compared to a Normal distribution. You can observe the skew as the 'S' shaped curve in the quantiles values.  

How does adding a trend change the ACF and PACF? Run the code in the cell below and examine the results. 

In [ ]:
auto_partial_corr_plot(trends)

Note that the ACF decays slowly, as was the case with the random walk. In addition, the PACF shows significant values for several lags. This is the result of the trend creating dependency from one value to the next. Any time series with a trend is **not stationary**.

### Time Series with a Seasonal Component

Many real-world time series include a seasonal component. A seasonal component is a period variation in the values of the time series. The periods can be measured in years, months, days, days of the week, hours of the day, etc. Some examples of seasonal components of time series include:

- Option expiration dates in capital markets.
- Annual holidays which can affect transportation, utility use, shopping habits, etc.
- Weekend vs. business days, which account for volumes of certain transaction behavior.
- Month of the year which can affect employment statistics, weather, etc.   
- Super Bowl Sunday. 
- Sunspot cycles.   

Let's investigate the properties of a time series with a seasonal component. The code in the cell below creates and plots a time series with the sinusoidal seasonal component, a trend, and added White noise. Run this code and examine the results.

In [ ]:
nr.seed(5544)
def seasonal_ts(start = '1-1990', end = '1-2015', freq = 'M', slope = 0.02, sd = 0.5, mean = 0):
    dates = pd.date_range(start = start, end = end, freq = freq)
    seasonal = pd.Series([slope*x for x in range(len(dates))],
                    index = dates)
    seasonal = seasonal + nr.normal(loc = mean, scale = sd, size = len(dates))
    seasonal = seasonal + [2.0*sin(pi*x/6) for x in range(len(dates))] + 5.0
    return(seasonal)

seasonal = seasonal_ts()
plot_ts(seasonal, 'seasonal data')

As expected, the time series looks like a noisy sine wave with a trend.

> **Exercise 32-3:** What does the distribution of values of the seasonal time series look like? What about the ACF and PACF of the seasonal time series? In the cell below, create and execute the code to examine the probability distribution, ACF, and PACF of this time series. 

In [ ]:
## Put your code below



> Answer the following questions:
> 1. How close to Normally distributed are the values of the seasonal time series? 
> 2. What do the ACF and PACF tell you about the periodicity of the series?      
> 3. Does the significance of the periodic behavior of the acf and pacf change with lag?  

> **Answers:**    
> 1.           
> 2.               .     
> 3.          

## Stationarity       

As was previously mentioned, **a time series with statistical properties that are invariant in time is considered stationary**. There are a number of ways to determine if a time series is stationary. Conversely, one can be certain that a time series is not stationary if it has any of the following properties:     
1. Random walk.    
2. Trend.   
3. Seasonality.   
4. Non-constant variance.   

Note, however, that having a stationary series does not preclude the presence of serial correlations. Do not confuse these points! Many time series models focus on the serial correlation properties of time series but require stationarity.   

A simple model for a time series process with white noise can be written:   

\begin{align}
y_t &= \phi y_{t-1} + w_t\\
w_t &\sim \mathbb{N}(0,\sigma)
\end{align}

The above is an example of an **autoregressive model**, where the next value at the next time step depends on the current value. We will revisit these models in the next chapter.   

The change from time $t-1$ to time $t$ can easly be worked out by taking the differences: 

$$\Delta y_t = (1- \phi) y_{t-1} + w_t$$

The foregoing equation has a root at $\phi = 1$, known as the **unit root**. At the root the difference between the difference between $y_t$ and $y_{t-1}$ is just $w_t$, giving rise to a random walk, which is stochastic and **not stationary**:   

$$y_t = \sum_{i=1}^t w_i$$ 

This example gives us a way to create hypothesis tests for stationarity. We can test the hypothesis that there is a unit root to determine if a time series is stationary or not. However, there is another consideration we must deal with. There are, in fact, several ways to define a model for a stationary process:    

1. A **unit root test** as discused above.   

$$y_t = \phi y_{t-1} + w_t$$

2. A **unit root test with a constant**, which includes a constant value in the formulation. Often this constant is some initial value, $c=y_0$, or a mean value. 

$$y_t = c + \phi y_{t-1} + w_t$$

3. A **Trend stationary process**, which may or may not include a constant. This version is used to test if a process is **stationary about a deterministic trend**, $\beta$. 

$$y_t = c + \beta t + \phi y_{t-1} + w_t$$

Given the foregoing, one can see that it is **important to correctly specify the correct model** for the case you are investigating! Failure to do so will result in erroneous results.    

There is another significant issue one must be aware of when working with unit root tests. There can be only a small difference between a time series with a unit root, which is non-stationary, and a time series with a root close to a unit, which is a stationary time series. As a result, unit root tests are said to **lack power**. This lack of power can have serious implications, since a hypothesis test may not be able to reject a hypothsis of non-stationary, when the time series in fact is, or in other cases, the oposite might be true. In practice, it is best to perform a visual inspection of the properties of the time series and well as perform unit root tests.   

We will work with two of the many possible tests here:            

1. A commonly used test is the [**Augmented Dicky-Fuller test**](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test) or **ADF** test. The ADF tests are unit root tests of the significance of a linear time series model. Coefficients represent components of the time series, such as trend and lagged differences, that make the series non-stationary. The **null distribution is that the series is non-stationary**.       

2. The [**Kwiatkowski–Phillips–Schmidt–Shin (KPSS) test**](https://en.wikipedia.org/wiki/KPSS_test) is a unit root test for stationarity. The **null hypothesis is that the time series is stationary**. The KPSS test can inlcude a trend termm, wherein the null hypothesis is that the series is trend-stationary.    

For the examples here, we will focus on the unit root with a constant and unit root with the trend and a constant since Statsmodels supports both options for the foregoing tests.          

Execute the code in the cell below which applies the ADF and KPSS tests to the series created above using two regression models for the tests.    

In [ ]:
def test_stationarity(series, regression_model='c', rounding=3):
    """Function to test the stationarity of time series using:
    1. Augmented Dicky Fuller test using AIC 
    2. KPSS test for constant
    
    Arguments:  
        Series: the time series to be evaluated.   
        regression_model: model to be used:   
            'c' - model with a constant   
            'ct' - model with a constant and trend  
        rounding: the number of digits to display in printed output.   
    """
    if regression_model not in ['c','ct']:    
        print('Error!: argument regression model must be one of {\'c\',\'ct\'}')
        return
    with warnings.catch_warnings(): 
        warnings.filterwarnings("ignore")
        kpss_test = kpss(series, regression=regression_model)
        ADF_test = adfuller(series, regression=regression_model, autolag="AIC")
    return [round(ADF_test[0],rounding), round(ADF_test[4]['5%'],rounding), round(ADF_test[1],rounding),
                                       round(kpss_test[0],rounding), round(kpss_test[1],rounding)]

def stationary_tests(series, names, regression_model='c', rounding=3):
    test_columns = ['ADF_statistic','ADF_conf_int','ADF_p_value','KPSS_statistic','KPSS_p_value']
    out = pd.DataFrame(index=names, columns=test_columns)    
    for ser, name in zip(series,names):  
        out.loc[name,:] = test_stationarity(ser, regression_model=regression_model, rounding=rounding)
    return out
        

time_series = [white,ts_periodic,walk,trends,seasonal]
names = ['white','ts_periodic','walk','trends','seasonal']

for model in ['c','ct']:
    print('\nFor model ' + model)
    print(stationary_tests(time_series, names, regression_model=model))  

> **Exercise 32-4:** Examine these resutls and answer the following questions:    
> 1. For the constant model, given the value of the ADF and KPSS statistics and the p-values, for which of these time series can we reject the null hypothesis of non-stationarity and why? Pay particular attention to the differences between the ADF and KPSS tests.     
> 2. For the trend plus constant model, given the value of the ADF and KPSS statistics and the p-values, for which of these time series can we reject the null hypothesis of non-stationarity and why? Pay particular attention to the differences between the ADF and KPSS tests.     
> 3. Which series can we conclude is non-statonary for all tests and why does this result make sense?     
> 4. As was discussed above, unit root tests have low power. Based on p-values and known properties of the time series, which cases can you identify where poor, or unexpected, inferences would result?   

> **Answers:** 
> 1.              
> 2.              
> 3.            
> 4.             

In the next two sections, we will explore decomposition methods for time series. The goal of these methods is to separate the non-stationary components and leave a stationary residual. The residual may have a significant serial correlation. In practice, this means that the trend and seasonal components must be estimated. Then the variance must be stabilized, so the variance is constant. 

To achieve these goals several methods can be used:     
1. Apply transformations to the variables, so that they are approximately Normally distributed with a constant variance in time.      
2. Decompose the time series into a trend, seasonal component, and residual.     
3. Use differencing operators to create a stationary product.    
4. Spectral decomposition of the time series into stationary frequency components. This method is discussed in a subsequent chapter.    

## STL Decomposition of Time Series

We have looked at the properties of several types of time series. 

- White noise series.
- Random walks.
- White noise series with trend.
- White noise series with seasonal component.

Next, we have to look into methods for decomposing time series data into its **trend, seasonal, and residual** components. This decomposition is necessary to remove nonstationary components from a time series so that further analysis can be performed. Failure to remove these components will result in poor models since the stationarity assumptions are violated.

A direct decomposition model is known as the **seasonal, trend and residual** or **STL** model. Not too surprisingly this model does the following:

- The trend is removed using a LOESS regression model. 
- The seasonal component is removed using a regression on periodic components.
- The remainder is known as the residual. 

The decomposition can be either **additive** or **multiplicative**. The additive model simply sums the components and is written:

$$TS(t) = S(t) + T(t) + R(t)$$

The multiplicative model multiplies the three components. This model is particularly useful in the common case where the **seasonal effect increases in proportion to the trend**. The multiplicative model is expressed:  

$$TS(t) = S(t)\ *\ T(t)\ *\ R(t)$$

Additive models are linear in their components and are therefore easier to work with. We can transform the multiplicative model into an additive model by a logarithm of both sides. The result is another additive model:  

\begin{align}
log(TS(t)) &= log(S(t)) + log(T(t)) + log(R(t)) \\
&= S^l(t) + T^l(t) + R^l(t)
\end{align}

You can find additional details of this model in [Rob Hyndman's lecture notes](http://robjhyndman.com/uwafiles/5-Cross-validation.pdf).    

Notice that there is no term in the STL model for a random walk. For random walk components, differencing operators are applied, which are addressed in the next section. 

### An STL decomposition example

Let's try this out on a time series which has a seasonal, trend and white noise residual component, using [statsmodels.tsa.seasonal.seasonal_decompose](https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html). The algorithm requires that we provide the **period of the seasonal component**. The code in the cell below models the trend and a monthly seasonal component. Run the code to compute the model and examine the results.

In [ ]:
def decomp_ts(ts, freq = 'M', model = 'additive'):
    res = sts.seasonal_decompose(ts, model = model) 
    res.plot()
    return(pd.DataFrame({'resid': res.resid, 
                         'trend': res.trend, 
                         'seasonal': res.seasonal},
                       index = ts.index) )

decomp = decomp_ts(seasonal) 
print(decomp[:12])
decomp[-12:]

You can see the time series is now decomposed into the three components. Notice that the first and last 6 values of the trend and seasonal component are missing. These missing values arise from the estimation of the seasonal component which must be truncated at the ends of the time series. We must take these missing values into account when performing any analysis. 

It appears that the STL process has removed most of the trend and seasonal components of the time series for the most part. We can do further analysis to understand these results. To do so, execute the code in the cell below to display some evaluation plots and summary statistics for the residual series. Notice that the first and last 6 samples are censored from the residual series which contain `nan` values.    

In [ ]:
def evaluate_resids(residual_series, title='residual series', regression_model='c', ljungbox=True, ADF=True):
    dist_ts(residual_series, title) 
    auto_partial_corr_plot(residual_series)
    if(ljungbox==True): print(acorr_ljungbox(residual_series, lags=40, model_df=0))
    if(ADF==True): print(stationary_tests([residual_series], [title], regression_model=regression_model))
                
        
evaluate_resids(decomp[6:-6].resid)    

Notice the following observations about these results:   
1. There is some significant autocorrelation, as evidenced by the Ljung Box test, so the residuals are not white noise.    
2. The residuals appear to be close to Normally distributed.     
3. The ADF and KPSS tests indicate that the residuals are stationary.   

### Another STL Example      

We will now try an example using some real-world data. The data are for the production of certain dairy products in the State of California, compiled by the California Department of Agriculture. To load these data and display the head of the data frame and plots of the milk produciton and log milk production.     

In [ ]:
CA_dairy = pd.read_csv('../data//cadairydata.csv')
CA_dairy.index = pd.to_datetime('1 ' + CA_dairy['Month.Number'].astype(str) + ' ' + CA_dairy['Year'].astype(str), format='%d %m %Y')
CA_dairy['LogMilkProd'] = np.log(CA_dairy['Milk.Prod'])

_,ax = plt.subplots(3,1, figsize=(10,9))
cols =['Milk.Prod','LogMilkProd','Icecream.Prod']
titles = ['Milk production time series','Log milk production time series','Ice cream production time series']
for ax_i, col, title in zip(ax, cols, titles):
    CA_dairy[col].plot(title=title, ax=ax_i)
    
CA_dairy    

There are several key observations that are important when working with these time series.   
1. None of these series are stationary. At the minimum, both the milk and ice cream series exhibit seasonal behavior. There is a strong trend in milk production and a possible trend in ice cream production.     
2. The variance of the milk production series is not constant. It appears that the log transformation of this series stabilizes the variance.
3. The seasonality of the ice cream series is close to constant over time. An additive model should be appropriate for this series.   

> **Exercise: 32-5:** You will now apply STL decomposition to the log milk time series and evaluate the results by the following steps:    
> 1. Compute the decomposition of the log milk time series, display the results, and save them to a variable `decomp_milk`.

In [ ]:
## Put your code below    



> 2. Use the `evaluate_resids` function to the residuals to evaluate the decomposition of the time series.

In [ ]:
## Put your code below  


> Examine these results and answer the following questions:     
> 1. Do the ACF, PACF, and Ljung-Box tests show evidence of serial correlation in the residuals, and how do these values compare to the original series?    
> 2. Is the distribution of the residuals close to a Normal distribution?      
> 3. Based on the ADF and KPSS statistics and p-values are the residuals stationary and why?  

> **Answers:**    
> 1.         
> 2.            
> 3.             

### Example: Decomposing the time series

We will now use linear models to perform a time series decomposition of the log of the milk time series. This decompositon will be performed by the following steps.     
1. Use the trend computed with the LOESS algorthm from the STL decomposition to remove the trend from the series.
2. Compute 12 monthy adjustments for the seasonality to remove the seasonality from the series.
3. Evaluate the stationarity and statistical properties of the residuals.

As a first step, execute the code in the cell below to detrend the time series and display the results.  

In [ ]:
CA_dairy['LogMilkProdNoTrend'] = CA_dairy['LogMilkProd'][6:-6] - decomp_milk[6:-6].trend
CA_dairy['month'] = CA_dairy.index.month
plot_ts(CA_dairy['LogMilkProdNoTrend'][6:-6], 'log milk production with no trend')

It appears that the time series has been succesfully detrended.   

The next step is to find the montly adjustment factor using a linear model. Execute the code in the cell below to fit a linear model and display the summary.  

In [ ]:
import statsmodels.formula.api as smf
seasonal_model = smf.ols(formula = 'LogMilkProdNoTrend ~ -1 + C(month)', data=CA_dairy).fit()
seasonal_model.summary()

The model is a bit overfit with some mothly adjustments that are not statistically significant. One could justify setting these coeffcients to 0. In this case, we will just use the estimates, since they are all small numbers.     

> *Note:* The OLS model used to compute the seasonal adjustments does not account for the seasonal serial correlation. The presence of serial correlation will inflate the variance of the model beyond the iid estimate. As a result, the use of an OLS model to estimate seasonal adjustments is an approximation.      

Next, execute the code in the cell below to apply the adjustments to the time series and examine the evaluation results. This is easily done by using the `predict` method of the linear model.   

In [ ]:
CA_dairy['LogMilkProdNoTrendNoSeason'] = CA_dairy['LogMilkProdNoTrend'] - seasonal_model.predict(CA_dairy) 
plot_ts(CA_dairy['LogMilkProdNoTrendNoSeason'][6:-6], 'log milk production with detrended and deseasonalized')
evaluate_resids(CA_dairy['LogMilkProdNoTrendNoSeason'][6:-6])  

Examining the above results we can draw the following conclusions:     
1. The ADF test and the KPSS test all indicate that the residuals are stationary.      
2. The distirbuiton of the residuals is nearly Normal.     
3. The ACF and PACT and the Ljung-Box Q statistic test show that the residuals still have information that can be used to model the detrended and deserialized time series.     
4. The ACF and PACF show some significant vlaues and lags around 6. This is likely the result of using a model that ignores serial correlation to estimate the seasonal adjustment.   
5. The small value of the Durban-Watson statistic ($<2$) in the model summary and results of the  indicate that the residuaals do have serial correlaiton, indicating there is information in the residuals after detrending and deserializing.     

## Differencing operators and random walks

We have just seen how the trend and seasonal components of a time series can be found using a decomposition model. Is there an alternative? Yes, **difference operators** can be used to remove seasonal, and random walk components. Deterministic and stochastic trend components are modeled using other methods, as is discussed in the next chapter.     

Difference operators can be over any number of lags. Lag 1 difference operators can be applied to remove random walks. Seasonal components can be removed by taking differences in the seasonal period. Further, we can apply difference operators more than once to the same series. For example, one can apply the following process:      
> 1. Apply difference operators of the seasonal period to remove the seasonal components.       
> 2. Test for stationarity of the resulting series.    
> 3. Apply another first difference operator to remove random walk components.    
> 4. Test for stationarity of the resulting series.   

For a random walk, a **first order difference operator** transforms the time series to a series of the **innovations or shocks**. An example will illustrate this point. Consider the plot below of the first-order differences of the random walk time series we explored earlier. The differences are computed using the Pandas [diff](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html) function. A first order difference series is one time-step shorter than the original series. Therefore the first sample is removed from the difference series with the *[1:]* operator. 

In [ ]:
walk_diffs = walk.diff()[1:]
_=plot_ts(walk_diffs, 'differences of random walk')

The difference series looks rather, random. What are the statistical properties? Is this now white noise? To find out, we will plot the ACF and PACF using the code in the cells below. 

> **Exercise 32-6:** The question is what are the statistical properties of the remainder series the first order differencing operator has been applied. Display of the resulting series using the `evaluate_resids`. **Hint:** You will need to remove the first value from the difference series since the difference operator cannot work on the first element of a time series. 

In [ ]:
## Put your code below



> Examine your results and answer the following questions:
> 1. Do the ACF, PACF, and Ljung-Box test show evidence of serial correlation in the residuals, and how do these values compare to the original series?    
> 2. Is the distribution of the residuals close to a Normal distribution?      
> 3. Based on the ADF and KPSS statistics and p-values are the residuals stationary and why?  

> **Answers:**      
> 1.             
> 2.          
> 3.          

### Example: Differencing the milk production time series        

We have already observed that the log-milk production time series has both trend and seasonality. We will now explore if the seasonality can be modeled with a differencing operator. A second question is if the resulting series is stationary.      

To start, execute the code in the cell below to perform the following operations on the log milk production series:     
1. Plot the log milk production time series.      
2. Display ACF and PACF plots.   
3. Compute and display the ADF and KPSS tests.    

Execute this code. 

In [ ]:
_,ax = plt.subplots(figsize=(8,3))
CA_dairy['LogMilkProd'].plot(ax=ax)
auto_partial_corr_plot(CA_dairy['LogMilkProd'])
stationary_tests([CA_dairy['LogMilkProd']], ['Log Milk Production Series'])

Examine these results, noticing the following:     
1. Not surprisingly, given the trend and seasonal behavior, the ADF and KPSS tests indicate that the series is not stationary. Further, the ACF shows the effect of the trend and the PACF has peaked at multiples of 12 lags.    
2. The variance of the log-transformed series appears to be fairly constant.    

### Example of trend and seasonal differencing   

You will now apply trend and seasonal differenceing to the log milk time series. This operation involves using two difference operators.    
1. $\nabla = x_t - x_{t-1}$ is a **stride one difference** operator.
2. $\nabla_S = x_t - x_{t-S}$ is the differnce operator for **seasonal period stride, S**.

To apply the product of these operators $\nabla \nabla_S$ we need to apply some algebra, to get a stationary series, $W_t$.             
\begin{align}
W_t &= \nabla \nabla_S y_t = \nabla_S y_t - \nabla_S y_{t-1} \\
&= (y_t - y_S) - (y_{t-1} - y_{S+1})
\end{align} 

For annual seasonal 12 month period, for this relationship becomes.

$$W_t = (y_t - y_{12}) - (y_{t-1} - y_{13})$$

> **Exercise 32-7:** You will now determine if seasonal differencing can provide a reasonable. To get started get these steps:     
> 1. Compute a difference series using the 12 month period of the seasonal behavior as an argument to the [pandas.DataFrame.diff](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html) method. The difference is then differenced again by subtracting the difference series less the first value (use the [1:] slice operator) for a back-shift of 1, from the difference series less the last value (use the [:-1] slide operator).           
> 2. Display a time series plot of the difference series.      
> 3. Apply the `evaluate_resids` function to the difference series.    
> Execute your code.    

In [ ]:
## Put your code below







> Carefully examine your results and answer these questions:  
> 1. Do the ACF, PACF, and Ljung-Box tests show evidence of serial correlation in the residuals, and how do these values compare to the original series?     
> 2. Is a component of the serial correlation seasonal and why?         
> 3. Is the distribution of the residuals close to a Normal distribution?      
> 4. Based on the ADF and KPSS statistics and p-values are the residuals stationary and why?   

> **Answers:**     
> 1.             
> 2.         
> 3.            
> 4.           

#### Copyright 2018, 2019, 2020, 2021, 2022, 2023, 2024 Stephen F Elston. All rights reserved.  